In [1]:
# Model Arguments 
from config import ModelArguments

config = ModelArguments()

# Multimodal Encoder, Resampler, Projector
from multimodal_encoder.builder import build_vision_tower
from multimodal_resampler.builder import build_vision_resampler
from multimodal_projector.builder import build_vision_projector

vision_tower = build_vision_tower(config)
vision_resampler = build_vision_resampler(config)
vision_projector = build_vision_projector(config)

On MacOS, decord package is not compatible with new python version.


/opt/homebrew/anaconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
from llava_llama import LlavaLlamaForCausalLM # Register the llava models into 'transformers'
from transformers import AutoConfig
from transformers import AutoModelForCausalLM

config = AutoConfig.for_model("llava_llama", trust_remote_code=True)
model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)

: 

In [2]:
# Dataset Construction for MultiModality
from dataprocess import LazySupervisedDataset, DataCollatorForSupervisedDataset
from torch.utils.data import DataLoader
from config import DataArguments 
from llava_llama import prep_llava_llama_tokenizer

data_args = DataArguments(
    data_path = "data/mock.json",
    image_folder = "data/",
    video_folder = "data/",
    video_fps = 1,
    frames_upbound = 0,
    add_time_instruction = False,
    force_sample = False,
    default_fps = 10
)

tokenizer = prep_llava_llama_tokenizer("meta-llama/Meta-Llama-3.1-8B-Instruct")
dataset = LazySupervisedDataset(data_args=data_args, tokenizer=tokenizer, image_processor=vision_tower.image_processor)
collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
dataloader = DataLoader(dataset, collate_fn=collator, batch_size=2, num_workers=1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from train import train 

train(model, tokenizer, dataloader, use_lora=True) # This is easily the better choice (so many lower-level optimization happens here ...)


In [ ]:
# Train on MoonDream fine-tune dataset

from torch.utils.data import Dataset
from datasets import load_dataset

class CaptchaDataset(Dataset):
    def __init__(self, split='train'):
        self.data = load_dataset("google/docci", trust_remote_code=True)[split]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return {
            "image": sample["image"], # Should be a PIL image
            "qa": [
                {
                    "question": "Describe this image.",
                    "answer": sample["description"],
                }
            ]
        }

datasets = {
    "train": CaptchaDataset("train"),
    "test": CaptchaDataset("test"),
}